In [1]:
# 공통
import tensorflow as tf
import numpy as np
import os

# 일관된 출력을 위해 유사난수 초기화
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# 맷플롯립 설정
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# 한글출력
plt.rcParams['font.family'] = 'NanumBarunGothic'
plt.rcParams['axes.unicode_minus'] = False

---
# CHAPTER 10. 인공 신경망 소개

---
## 10.1 생물학적 뉴런에서 인공 뉴런까지

### 10.1.1 생물학적 뉴런

![Figure10-1](./images/Figure10-1.png)
**<center>그림 10-1 생물학적 뉴런</center>**

+ **핵**<sup>Nucleus</sup>
+ **세포체**<sup>Cell body</sup>
+ **수상돌기**<sup>Dendrite</sup>
+ **축삭돌기**<sup>Axon</sup>
+ **축삭끝가지**<sup>Telodendria</sup>
+ **시냅스 말단**<sup>Synaptic terminals</sup>(또는 간단히 **시냅스**<sup>Synapse</sup>)
+ 생물학적 뉴런은 이런 시냅스를 통해 다른 뉴런으로부터 짧은 전기 자극 **신호**<sup>signal</sup>를 받음
+ 개개의 생물학적 뉴런은 단순하게 동작하지만, 수십억 개의 뉴런이 네트워크로 조직되어 있음

![Figure10-2](./images/Figure10-2.png)
**<center>그림 10-2 생물학적 신경망의 여러 층(인간 피질)</center>**

### 10.1.2 뉴런을 사용한 논리 연산

![Figure10-3](./images/Figure10-3.png)
**<center>그림 10-3 간단한 논리 연산을 수행하는 인공 신경망</center>**

### 10.1.3 퍼셉트론<sup>Perceptron</sup>

+ **TLU**<sup>threshold logic unit</sup> 형태의 인공 뉴런을 기반으로 함
+ 입력과 출력이 이진 on/off 값이 아니라 숫자 값

![Figure10-4](./images/Figure10-4.png)
**<center>그림 10-4 TLU</center>**

![Equation10-1](./images/Equation10-1.png)
**<center>식 10-1 퍼셉트론에서 일반적으로 사용하는 계단 함수</center>**

![Figure10-5](./images/Figure10-5.png)
**<center>그림 10-5 퍼셉트론 다이어그램</center>**

![Equation10-2](./images/Equation10-2.png)
**<center>식 10-2 퍼셉트론 학습 규칙(가중치 업데이트)</center>**

+ $w$<sub>$i,j$</sub>는 $i$번째 입력 뉴런과 $j$번째 출력 뉴런 사이를 연결하는 가중치
+ $x$<sub>$i$</sub>는 현재 훈련 샘플의 $i$번째 뉴런의 입력값
+ $\hat{y}$<sub>$j$</sub>는 현재 훈련 샘플의 $j$번째 출력 뉴런의 출력값
+ $y$<sub>$j$</sub>는 현재 훈련 샘플의 $j$번째 출력 뉴런의 타깃값
+ $\eta$는 학습률

+ **퍼셉트론 수렴 이론**<sup>Perceptron convergence theorem</sup> : 훈련 샘플이 선형적으로 구분될 수 있다면 정답에 수렴함

In [2]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

iris = load_iris()
X = iris.data[:, (2, 3)]  # 꽃잎 길이, 꽃잎 너비
y = (iris.target == 0).astype(np.int)

per_clf = Perceptron(max_iter=100, random_state=42)
per_clf.fit(X, y)

y_pred = per_clf.predict([[2, 0.5]])
y_pred

C:\Users\link\Anaconda3\envs\mlbook\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


array([1])

![Figure10-6](./images/Figure10-6.png)
**<center>그림 10-6 XOR 분류 문제와 이를 푸는 다층 퍼셉트론</center>**

### 10.1.4 다층 퍼셉트론(MLP)과 역전파

![Figure10-7](./images/Figure10-7.png)
**<center>그림 10-7 다층 퍼셉트론</center>**

+ **은닉 층**<sup>hidden layer</sup>이 2개 이상일 때 **심층 신경망**<sup>deep neural network</sup>(DNN)이라고 함

![Figure10-7-1](./images/Figure10-7-1.png)
**<center>그림 10-7-1 Backpropagation</center>**

+ 계단 함수에는 수평선밖에 없어서 계산할 그래디언트가 없음  
→ 알고리즘을 잘 작동시키기 위해 계단 함수 대신 **활성화 함수**<sup>activation function</sup>를 사용

![Figure10-8](./images/Figure10-8.png)
**<center>그림 10-8 활성화 함수와 해당 도함수</center>**

+ 각 출력이 서로 다른 이진 클래스에 대응되는 분류 문제에 자주 사용됨
+ 클래스가 배타적일 때는 전형적으로 출력층의 활성화 함수를 **소프트맥스**<sup>softmax</sup> 함수를 사용

![Figure10-9](./images/Figure10-9.png)
**<center>그림 10-9 분류에 사용되는 (ReLU와 소프트맥스를 포함한) 현대적 다층 퍼셉트론</center>**

+ **피드포워드 신경망**<sup>feed forward neural network</sup>(FNN) : 신호가 한 방향으로만 흐르는 구조의 신경망

---
## 10.2 텐서플로의 고수준 API로 다층 퍼셉트론 훈련하기

In [2]:
import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train.shape, y_train.shape ,X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [3]:
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
X_train.shape, X_test.shape

((60000, 784), (10000, 784))

In [4]:
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [6]:
feature_cols = [tf.feature_column.numeric_column("X", shape=[28 * 28])]
dnn_clf = tf.estimator.DNNClassifier(
    hidden_units=[300,100],
    n_classes=10,
    feature_columns=feature_cols
)

input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_train},
    y=y_train,
    num_epochs=40,
    batch_size=50,
    shuffle=True
)

dnn_clf.train(input_fn=input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\link\\AppData\\Local\\Temp\\tmp11s3czce', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000026BD3244080>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for upd

INFO:tensorflow:global_step/sec: 294.039
INFO:tensorflow:loss = 0.3755817, step = 6000 (0.338 sec)
INFO:tensorflow:global_step/sec: 349.363
INFO:tensorflow:loss = 12.512836, step = 6100 (0.287 sec)
INFO:tensorflow:global_step/sec: 342.209
INFO:tensorflow:loss = 0.84577435, step = 6200 (0.292 sec)
INFO:tensorflow:global_step/sec: 302.01
INFO:tensorflow:loss = 1.1087593, step = 6300 (0.331 sec)
INFO:tensorflow:global_step/sec: 343.382
INFO:tensorflow:loss = 7.1294475, step = 6400 (0.291 sec)
INFO:tensorflow:global_step/sec: 353.054
INFO:tensorflow:loss = 0.27368137, step = 6500 (0.282 sec)
INFO:tensorflow:global_step/sec: 350.584
INFO:tensorflow:loss = 0.6790308, step = 6600 (0.285 sec)
INFO:tensorflow:global_step/sec: 329.826
INFO:tensorflow:loss = 0.5001973, step = 6700 (0.303 sec)
INFO:tensorflow:global_step/sec: 302.922
INFO:tensorflow:loss = 0.53347594, step = 6800 (0.331 sec)
INFO:tensorflow:global_step/sec: 337.601
INFO:tensorflow:loss = 0.11546424, step = 6900 (0.295 sec)
INFO:te

INFO:tensorflow:global_step/sec: 351.816
INFO:tensorflow:loss = 0.07563539, step = 14200 (0.285 sec)
INFO:tensorflow:global_step/sec: 344.559
INFO:tensorflow:loss = 7.3370123, step = 14300 (0.290 sec)
INFO:tensorflow:global_step/sec: 338.741
INFO:tensorflow:loss = 0.19483253, step = 14400 (0.295 sec)
INFO:tensorflow:global_step/sec: 336.468
INFO:tensorflow:loss = 0.028300187, step = 14500 (0.296 sec)
INFO:tensorflow:global_step/sec: 337.599
INFO:tensorflow:loss = 0.72543234, step = 14600 (0.296 sec)
INFO:tensorflow:global_step/sec: 324.49
INFO:tensorflow:loss = 0.11211136, step = 14700 (0.309 sec)
INFO:tensorflow:global_step/sec: 322.403
INFO:tensorflow:loss = 0.12904906, step = 14800 (0.310 sec)
INFO:tensorflow:global_step/sec: 330.915
INFO:tensorflow:loss = 0.15222552, step = 14900 (0.301 sec)
INFO:tensorflow:global_step/sec: 315.305
INFO:tensorflow:loss = 0.29641616, step = 15000 (0.318 sec)
INFO:tensorflow:global_step/sec: 333.114
INFO:tensorflow:loss = 0.19041187, step = 15100 (0.

INFO:tensorflow:global_step/sec: 295.774
INFO:tensorflow:loss = 0.14736281, step = 22300 (0.339 sec)
INFO:tensorflow:global_step/sec: 338.74
INFO:tensorflow:loss = 0.04170479, step = 22400 (0.294 sec)
INFO:tensorflow:global_step/sec: 350.584
INFO:tensorflow:loss = 0.04132849, step = 22500 (0.286 sec)
INFO:tensorflow:global_step/sec: 323.443
INFO:tensorflow:loss = 0.10253537, step = 22600 (0.308 sec)
INFO:tensorflow:global_step/sec: 324.49
INFO:tensorflow:loss = 0.016451241, step = 22700 (0.308 sec)
INFO:tensorflow:global_step/sec: 350.585
INFO:tensorflow:loss = 0.0411376, step = 22800 (0.286 sec)
INFO:tensorflow:global_step/sec: 346.945
INFO:tensorflow:loss = 0.08194381, step = 22900 (0.288 sec)
INFO:tensorflow:global_step/sec: 345.749
INFO:tensorflow:loss = 0.031336386, step = 23000 (0.288 sec)
INFO:tensorflow:global_step/sec: 313.335
INFO:tensorflow:loss = 0.006905762, step = 23100 (0.320 sec)
INFO:tensorflow:global_step/sec: 316.3
INFO:tensorflow:loss = 0.07559003, step = 23200 (0.3

INFO:tensorflow:global_step/sec: 329.826
INFO:tensorflow:loss = 0.018173363, step = 30400 (0.302 sec)
INFO:tensorflow:global_step/sec: 321.369
INFO:tensorflow:loss = 0.053593688, step = 30500 (0.311 sec)
INFO:tensorflow:global_step/sec: 348.15
INFO:tensorflow:loss = 0.03936865, step = 30600 (0.288 sec)
INFO:tensorflow:global_step/sec: 335.342
INFO:tensorflow:loss = 0.007256469, step = 30700 (0.297 sec)
INFO:tensorflow:global_step/sec: 335.342
INFO:tensorflow:loss = 0.021481397, step = 30800 (0.298 sec)
INFO:tensorflow:global_step/sec: 346.946
INFO:tensorflow:loss = 0.008789353, step = 30900 (0.288 sec)
INFO:tensorflow:global_step/sec: 341.044
INFO:tensorflow:loss = 0.03036324, step = 31000 (0.293 sec)
INFO:tensorflow:global_step/sec: 305.693
INFO:tensorflow:loss = 0.020313986, step = 31100 (0.327 sec)
INFO:tensorflow:global_step/sec: 335.342
INFO:tensorflow:loss = 0.009918319, step = 31200 (0.299 sec)
INFO:tensorflow:global_step/sec: 351.816
INFO:tensorflow:loss = 0.023779236, step = 3

INFO:tensorflow:global_step/sec: 325.546
INFO:tensorflow:loss = 0.056337953, step = 38500 (0.307 sec)
INFO:tensorflow:global_step/sec: 348.15
INFO:tensorflow:loss = 0.020796543, step = 38600 (0.286 sec)
INFO:tensorflow:global_step/sec: 350.585
INFO:tensorflow:loss = 0.0076301303, step = 38700 (0.285 sec)
INFO:tensorflow:global_step/sec: 334.223
INFO:tensorflow:loss = 0.005468433, step = 38800 (0.300 sec)
INFO:tensorflow:global_step/sec: 327.671
INFO:tensorflow:loss = 0.026583642, step = 38900 (0.305 sec)
INFO:tensorflow:global_step/sec: 307.569
INFO:tensorflow:loss = 0.028222168, step = 39000 (0.324 sec)
INFO:tensorflow:global_step/sec: 349.363
INFO:tensorflow:loss = 0.0055801016, step = 39100 (0.286 sec)
INFO:tensorflow:global_step/sec: 327.67
INFO:tensorflow:loss = 0.012595018, step = 39200 (0.306 sec)
INFO:tensorflow:global_step/sec: 302.923
INFO:tensorflow:loss = 0.0015620094, step = 39300 (0.330 sec)
INFO:tensorflow:global_step/sec: 350.584
INFO:tensorflow:loss = 0.02903568, step 

In [7]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_test},
    y=y_test,
    shuffle=False
)

eval_results = dnn_clf.evaluate(input_fn=test_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-11-16:17:16
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\link\AppData\Local\Temp\tmp11s3czce\model.ckpt-44000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-11-16:17:16
INFO:tensorflow:Saving dict for global step 44000: accuracy = 0.9792, average_loss = 0.10099163, global_step = 44000, loss = 12.783751
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 44000: C:\Users\link\AppData\Local\Temp\tmp11s3czce\model.ckpt-44000


In [8]:
eval_results

{'accuracy': 0.9792,
 'average_loss': 0.10099163,
 'loss': 12.783751,
 'global_step': 44000}

---
## 10.3 텐서플로의 저수준 API로 심층 신경망 훈련하기

### 10.3.1 구성 단계

In [9]:
import tensorflow as tf
import numpy as np

n_inputs = 28*28  # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [10]:
with tf.name_scope("dnn"):
    _hidden1 = neuron_layer(X, n_hidden1, name="_hidden1", activation=tf.nn.relu)
    _hidden2 = neuron_layer(_hidden1, n_hidden2, name="_hidden2", activation=tf.nn.relu)
    _logits = neuron_layer(_hidden2, n_outputs, name="_outputs")

In [11]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    y_proba = tf.nn.softmax(logits)

In [12]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [13]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [14]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [15]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

### 10.3.2 실행 단계

In [16]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [17]:
n_epochs = 20
n_batches = 50
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "배치 데이터 정확도:", acc_batch, "검증 세트 정확도:", acc_valid)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 배치 데이터 정확도: 0.86 검증 세트 정확도: 0.9042
1 배치 데이터 정확도: 0.92 검증 세트 정확도: 0.9204
2 배치 데이터 정확도: 0.9 검증 세트 정확도: 0.9324
3 배치 데이터 정확도: 0.94 검증 세트 정확도: 0.9412
4 배치 데이터 정확도: 0.94 검증 세트 정확도: 0.943
5 배치 데이터 정확도: 0.88 검증 세트 정확도: 0.948
6 배치 데이터 정확도: 0.88 검증 세트 정확도: 0.9508
7 배치 데이터 정확도: 0.94 검증 세트 정확도: 0.9542
8 배치 데이터 정확도: 0.94 검증 세트 정확도: 0.9574
9 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9596
10 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9622
11 배치 데이터 정확도: 0.94 검증 세트 정확도: 0.9642
12 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9662
13 배치 데이터 정확도: 0.96 검증 세트 정확도: 0.9664
14 배치 데이터 정확도: 0.96 검증 세트 정확도: 0.9684
15 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9698
16 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9684
17 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9702
18 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9718
19 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9732


### 10.3.3 신경망 사용하기

In [18]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    X_new_scaled = X_test[:20]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


In [19]:
print("예측 클래스:", y_pred)
print("타깃 클래스:", y_test[:20])

예측 클래스: [7 2 1 0 4 1 4 9 6 9 0 6 9 0 1 5 9 7 3 4]
타깃 클래스: [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]


---
## 10.4 신경망 하이퍼파라미터 튜닝하기

+ 신경망의 유연성은 중요한 단점이 되기도 함 → 조절해야 할 하이퍼파라미터가 많아짐
+ 튜닝방법
    + 교차 검증을 활용한 그리드 탐색, 랜덤 탐색
    + 오스카<sup>Oscar</sup>(http://oscar.calldesk.ai/)

### 10.4.1 은닉층의 수

+ 은닉층이 하나인 다층 퍼셉트론이더라도 뉴런 수가 충분하면 아주 복잡한 함수도 모델링할 수 있음
+ 하지만 심층 신경망이 얕은 신경망보다 **파라미터 효율성**이 훨씬 좋음
+ 훈련 세트에 과대적합이 생기기 전까지 점진적으로 은닉층의 수를 늘릴 수 있음
+ 미리 훈련된 네트워크 일부를 **재사용**할 수 있음

### 10.4.2 은닉층의 뉴런 수

+ 입력층과 출력층의 뉴런 수는 해당 작업에 필요한 입력과 출력의 형태에 따라 결정됨
    + MNIST는 28×28=784개의 입력 뉴런과 10개의 출력 뉴런
+ 모든 은닉층에 같은 뉴런의 크기를 사용하면 하이퍼파라미터를 층마다 따로 설정하지 않아도 됨
+ 네트워크가 과대적합이 시작되기 전까지 점진적으로 뉴런 수를 늘릴 수 있음
+ 일반적으로 층의 뉴런 수보다 층 수를 늘리는 쪽이 이득이 많음
+ **스트레치 팬츠**<sup>stretch pants</sup> 방식 : 실제 필요한 것보다 더 많은 층과 뉴런을 가진 모델을 선택하고, 과대적합되지 않도록 조기 종료 기법을 사용

### 10.4.3 활성화 함수

+ 대부분의 은닉층에 **ReLU** 활성화 함수를 사용
    + 다른 활성화 함수보다 계산이 조금 더 빠름
    + 입력값이 클 때 특정 값에 수렴하지 않는 덕분에 경사 하강법이 평편한 지역에서 심하게 지체되지 않음
+ 클래스가 상호 배타적일 경우 출력층에서는 일반적으로 **소프트맥스** 함수를 사용
+ 회귀 작업일 경우에는 출력층에서 활성화 함수를 사용하지 않음

---
## 10.5 연습문제

#### 1. ([그림 10-3]에 있는 것과 같은) 초창기 인공 뉴런을 사용해 $A⊕B$(⊕는 XOR 연산입니다)를 계산하는 인공신경망을 그려보세요. 힌트: $A⊕B = (A ∧ ¬B) ∨ (¬A ∧ B).$

![Figure10-10](./images/Figure10-10.png)
**<center>그림 10-10 연습문제 1</center>**

#### 2. 고전적인 퍼셉트론(즉, 퍼셉트론 훈련 알고리즘으로 훈련된 단일 TLU)보다 로지스틱 회귀 분류기가 일반적으로 선호되는 이유는 무엇인가요? 퍼셉트론을 어떻게 수정하면 로지스틱 회귀 분류기와 동등하게 만들 수 있나요?

☞ 고전적인 퍼셉트론은 데이터셋이 선형적으로 구분될 때만 수렴하고 클래스 확률을 추정할 수 없습니다. 이와는 반대로 로지스틱 회귀 분류기는 데이터셋이 선형적으로 구분되지 못 해도 좋은 솔루션으로 수렴하고 클래스 확률을 출력합니다. 퍼셉트론의 활성화 함수를 로지스틱 활성화 함수로(또는 여러 개의 뉴런일 경우 소프트맥스 활성화 함수로) 바꾸고, 경사 하강법을 사용하여(또는 크로스 엔트로피 같은 비용 함수를 최소화하는 다른 최적화 알고리즘을 사용하여) 훈련시키면 로지스틱 회귀 분류기와 동일하게 됩니다.

#### 3. 왜 초창기의 다층 퍼셉트론을 훈련시킬 때 로지스틱 활설화 함수가 핵심 요소였나요?

☞ 로지스틱 활성화 함수의 도함수는 어디에서나 0이 아니어서 경사 하강법이 항상 경사를 따라 이동할 수 있으므로 초창기 MLP의 핵심 요소였습니다. 활성화 함수가 계단 함수일 때는 경사가 없기 때문에 경사 하강법이 이동할 수 없습니다.

#### 4. 유명한 활성화 함수 네 가지는 무엇인가요?

![Figure10-8](./images/Figure10-8.png)
**<center>그림 10-8 활성화 함수와 해당 도함수</center>**

#### 5. 10개의 통과 뉴런으로 된 입력층, 50개의 뉴런으로 된 은닉층, 그리고 3개의 뉴런으로 된 출력층으로 구성된 다층 퍼셉트론이 있다고 가정합시다. 모든 뉴런은 ReLU 활성화 함수를 사용합니다.
+ **입력 행렬 $X$의 크기는 얼마인가요?**  
☞ m×10 (m은 훈련 배치의 크기)
+ **은닉층의 가중치 벡터 $W$<sub>$h$</sub>와 편향 벡터 $b$<sub>$h$</sub>의 크기는 얼마인가요?**  
☞ 10×50, 50
+ **출력층의 가중치 벡터 $W$<sub>$o$</sub>와 편향 벡터 $b$<sub>$o$</sub>의 크기는 얼마인가요?**  
☞ 50×3, 3
+ **네트워크의 출력 행렬 $Y$의 크기는 얼마인가요?**  
☞ m×3
+ **$X$, $W$<sub>$h$</sub>, $b$<sub>$h$</sub>, $W$<sub>$o$</sub>, b<sub>$o$</sub>의 함수로 네트워크의 출력 행렬 Y를 계산하는 식을 써보세요.**  
☞ $Y$ = ReLU(ReLU($X$·$W$<sub>h</sub> + $b$<sub>h</sub>)·$W$<sub>$o$</sub> + $b$<sub>$o$</sub>)

#### 6. 스팸 메일을 분류하기 위해서는 출력층에 몇 개의 뉴런이 필요할까요? 출력층에 어떤 활성화 함수를 사용해야 할까요? MNIST 문제라면 출력층에 어떤 활성화 함수를 사용하고 뉴런은 몇 개가 필요할까요? 2장에서 본 주택 가격 예측용 네트워크에 대해 같은 질문의 답을 찾아보세요.

☞ 스팸 메일을 분류하기 위해서는 신경망의 출력층에 하나의 뉴런만 필요합니다. 예를 들어 이메일이 스팸일 확률을 출력합니다. 확률을 추정할 때 일반적으로 출력층에 로지스틱(Sigmoid) 활성화 함수를 사용합니다. MNIST 문제라면 출력층에 10개의 뉴런이 필요하고, 다중 클래스 환경에서 클래스마다 하나의 확률을 출력하기 위해 로지스틱 함수를 소프트맥스 활성화 함수로 바꾸어야 합니다. 2장에서처럼 주택 가격을 예측하는 신경망을 만들고 싶다면 출력층에 활성화 함수가 없는 출력 뉴런 하나가 필요합니다.

#### 7. 역전파란 무엇이고 어떻게 작동하나요? 역전파와 후진 모드 자동 미분의 차이점은 무엇인가요?

![Figure10-7-1](./images/Figure10-7-1.png)
**<center>그림 10-7-1 Backpropagation</center>**

+ **역전파** : 그래디언트 계산과 경사 하강법 스텝을 여러 번 수행하여 인공 신경망을 훈련시키는 전체 프로세스
+ **후진 모드 자동 미분** : 그래디언트를 효과적으로 계산하는 하나의 기법

#### 8. 다층 퍼셉트론에서 조정할 수 있는 하이퍼파라미터를 모두 나열해보세요. 훈련 데이터에 다층 퍼셉트론이 과대적합되었다면 이를 해결하기 위해 하이퍼파라미터를 어떻게 조정해야 할까요?

☞ 기본 MLP에서 바꿀 수 있는 하이퍼파라미터는 은닉층 수, 각 은닉층의 뉴런 수, 각 은닉층과 출력층에서 사용하는 활성화 함수입니다. 일반적으로 ReLU(또는 이 함수의 변종)가 은닉층의 활성화 함수 기본값으로 좋습니다. 출력층에서는 일반적으로 이진 분류에서는 로지스틱 활성화 함수, 다중 분류에서는 소프트맥스 활성화 함수를 사용하고 회귀에서는 활성화 함수를 적용하지 않습니다.  
☞ MLP가 훈련 데이터에 과대적합되었다면 은닉층 수와 각 은닉층에 있는 뉴런 수를 줄여볼 수 있습니다.

#### 9. 깊은<sup>deep</sup> 다층 퍼셉트론을 MNIST 데이터셋에 훈련시키고 98% 정확도를 얻을 수 있는지 확인해보세요. 9장의 마지막 연습문제에서와 같이 모든 부가 기능을 추가해보세요(즉, 체크포인트를 저장하고, 중지되었을 때 마지막 체크포인트를 복원하고, 서머리를 추가하고, 텐서보드를 사용해 학습 곡선을 그려보세요).

In [5]:
import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train.shape, y_train.shape ,X_test.shape, y_test.shape
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

n_inputs = 28*28  # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [6]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [7]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

In [8]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    loss_summary = tf.summary.scalar('log_loss', loss)

In [9]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [10]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    accuracy_summary = tf.summary.scalar('accuracy', accuracy)

In [11]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [12]:
from datetime import datetime

def log_dir(prefix=""):
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    root_logdir = "tf_logs"
    if prefix:
        prefix += "-"
    name = prefix + "run-" + now
    return "{}/{}/".format(root_logdir, name)

In [13]:
logdir = log_dir("mnist_dnn")

In [14]:
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [16]:
m, n = X_train.shape
m, n

(55000, 784)

In [19]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [20]:
import os

n_epochs = 10001
batch_size = 50
n_batches = int(np.ceil(m / batch_size))

checkpoint_path = "/tmp/my_deep_mnist_model.ckpt"
checkpoint_epoch_path = checkpoint_path + ".epoch"
final_model_path = "./my_deep_mnist_model"

best_loss = np.infty
epochs_without_progress = 0
max_epochs_without_progress = 50

with tf.Session() as sess:
    if os.path.isfile(checkpoint_epoch_path):
        # 체크포인트 파일이 있으면 모델을 복원하고 에포크 숫자를 로드합니다
        with open(checkpoint_epoch_path, "rb") as f:
            start_epoch = int(f.read())
        print("이전 훈련이 중지되었습니다. 에포크 {}에서 시작합니다".format(start_epoch))
        saver.restore(sess, checkpoint_path)
    else:
        start_epoch = 0
        sess.run(init)

    for epoch in range(start_epoch, n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val, loss_val, accuracy_summary_str, loss_summary_str = 
        sess.run([accuracy, loss, accuracy_summary, loss_summary], feed_dict={X: X_valid, y: y_valid})
        
        file_writer.add_summary(accuracy_summary_str, epoch)
        file_writer.add_summary(loss_summary_str, epoch)
        if epoch % 5 == 0:
            print("에포크:", epoch,
                  "\t검증 세트 정확도: {:.3f}%".format(accuracy_val * 100),
                  "\t손실: {:.5f}".format(loss_val))
            saver.save(sess, checkpoint_path)
            with open(checkpoint_epoch_path, "wb") as f:
                f.write(b"%d" % (epoch + 1))
            if loss_val < best_loss:
                saver.save(sess, final_model_path)
                best_loss = loss_val
            else:
                epochs_without_progress += 5
                if epochs_without_progress > max_epochs_without_progress:
                    print("조기 종료")
                    break

에포크: 0 	검증 세트 정확도: 89.980% 	손실: 0.35609
에포크: 5 	검증 세트 정확도: 95.080% 	손실: 0.17742
에포크: 10 	검증 세트 정확도: 96.260% 	손실: 0.13394
에포크: 15 	검증 세트 정확도: 97.020% 	손실: 0.10702
에포크: 20 	검증 세트 정확도: 97.400% 	손실: 0.09347
에포크: 25 	검증 세트 정확도: 97.640% 	손실: 0.08430
에포크: 30 	검증 세트 정확도: 97.600% 	손실: 0.07822
에포크: 35 	검증 세트 정확도: 97.740% 	손실: 0.07499
에포크: 40 	검증 세트 정확도: 97.760% 	손실: 0.07025
에포크: 45 	검증 세트 정확도: 97.880% 	손실: 0.06778
에포크: 50 	검증 세트 정확도: 97.820% 	손실: 0.06809
에포크: 55 	검증 세트 정확도: 98.060% 	손실: 0.06550
에포크: 60 	검증 세트 정확도: 98.080% 	손실: 0.06627
에포크: 65 	검증 세트 정확도: 98.120% 	손실: 0.06586
에포크: 70 	검증 세트 정확도: 98.060% 	손실: 0.06703
에포크: 75 	검증 세트 정확도: 97.980% 	손실: 0.06788
에포크: 80 	검증 세트 정확도: 98.040% 	손실: 0.06856
에포크: 85 	검증 세트 정확도: 98.120% 	손실: 0.06758
에포크: 90 	검증 세트 정확도: 98.160% 	손실: 0.06774
에포크: 95 	검증 세트 정확도: 98.200% 	손실: 0.06803
에포크: 100 	검증 세트 정확도: 98.040% 	손실: 0.06871
에포크: 105 	검증 세트 정확도: 98.200% 	손실: 0.06920
조기 종료


In [21]:
os.remove(checkpoint_epoch_path)

In [22]:
with tf.Session() as sess:
    saver.restore(sess, final_model_path)
    accuracy_val = accuracy.eval(feed_dict={X: X_test, y: y_test})

INFO:tensorflow:Restoring parameters from ./my_deep_mnist_model


In [23]:
accuracy_val

0.9794